In [1]:
%load_ext autoreload
%autoreload 2

In [59]:
from helpers import *
import numpy as np

In [60]:
data = [ 
    'data/social_cocktail.json',
    'data/liquor.json'
]

descriptions, names = [], []

for d in data:
    descriptions += load_data(d, field='description')
    names += load_data(d, field='name')

print('There are {} recipes in the database.'.format(len(descriptions)))

There are 1153 recipes in the database.


In [61]:
descriptions = [clean_string(x) for x in descriptions]
X_desc = flatten_list(descriptions)

names = [clean_string(x) for x in names]
X_names = flatten_list(names)

In [65]:
# Limit the vocabulary size
vocabulary_size=900
X_limit = limit_vocabulary(X_desc, vocabulary_size=vocabulary_size)

# One-hot encode the data
X_limit_transformed, label_encoder, onehot_encoder = encode_categorical(X_limit)
print('There are {} words with a vocabulary size of {}'.format(*X_limit_transformed.shape))

# Transform data into sequences and predictions
seq_length = 10
X, y = sequence_transform(X_limit_transformed, seq_length)
print('The sequence length is {}'.format(seq_length))
print('Observation shape is {}, label shape is {}'.format(X.shape, y.shape))

There are 31879 words with a vocabulary size of 900
The sequence length is 10
Observation shape is (31869, 10, 900), label shape is (31869, 900)


In [66]:
N = X.shape[0]
X_train, y_train = X[:int(N*0.95)], y[:int(N*0.95)]
X_val, y_val = X[int(N*0.95):], y[int(N*0.95):]

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(30275, 10, 900) (30275, 900) (1594, 10, 900) (1594, 900)


### Create the Neural Network Models

In [67]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [68]:
model = Sequential()
# model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(256, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))

model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'], optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 256)           1184768   
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 900)               116100    
Total params: 1,497,988
Trainable params: 1,497,988
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Create the model
batch_size = 32
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=batch_size)

Train on 30275 samples, validate on 1594 samples
Epoch 1/1
 7968/30275 [======>.......................] - ETA: 982s - loss: 4.8859 - categorical_accuracy: 0.1020

In [58]:
start = np.random.randint(0, len(X)-1)
seed = ''

for i in range(seq_length):
    x = X[start, i].reshape(1, -1)
    seed += reverse_encoding(x, label_encoder, onehot_encoder)[0] + ' '
    
seed = seed.strip()
observation = X[start]

result = []
str_len = 100

for i in range(str_len):
    prediction = predict_observation(
        model, np.array([observation]), 32, label_encoder, onehot_encoder, raw_prediction=True
    )
    result.append(prediction)    
    observation = np.vstack((observation[1:, :], prediction))
    
result_np = np.array(result).reshape(str_len, vocabulary_size)
txt = reverse_encoding(result_np, label_encoder, onehot_encoder)
print(seed, txt)

of lemon bitters and serve . fill rocks glass with ['ice' '.' 'add' 'all' 'ingredients' 'to' 'shaker' 'and' 'fill' 'with'
 'ice' '.' 'shake' 'and' 'strain' 'into' 'chilled' 'cocktail' 'glass' '.'
 'garnish' 'with' 'lemon' 'twist' '.' 'add' 'all' 'ingredients' 'to'
 'mixing' 'tin' 'and' 'fill' 'with' 'ice' '.' 'shake' 'and' 'strain' 'into'
 'chilled' 'cocktail' 'glass' '.' 'garnish' 'with' 'lemon' 'twist' '.'
 'add' 'all' 'ingredients' 'to' 'mixing' 'tin' 'and' 'fill' 'with' 'ice'
 '.' 'shake' 'and' 'strain' 'into' 'chilled' 'cocktail' 'glass' '.'
 'garnish' 'with' 'lemon' 'twist' '.' 'add' 'all' 'ingredients' 'to'
 'mixing' 'tin' 'and' 'fill' 'with' 'ice' '.' 'shake' 'and' 'strain' 'into'
 'chilled' 'cocktail' 'glass' '.' 'garnish' 'with' 'lemon' 'twist' '.'
 'add' 'all' 'ingredients']


In [29]:
model_file = 'model.json'
weights_file = 'weights.h5'

def save_model_to_json(model, model_file, weights_file):
    model_json = model.to_json()
    with open(model_file, "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weights_file)
    
save_model_to_json(model, model_file, weights_file)